In [7]:
from configparser import ConfigParser
from datetime import datetime
import pandas as pd
import pyodbc

In [8]:
# CHANGE THE DIRECTORY TO NOTEBOOK/NOTEBOOK
try:
    cd /notebooks/notebooks
except:
    %cd /notebooks/notebooks

/notebooks/notebooks


In [9]:
CurrentDate = str(datetime.now())
delimiter1 = " ";
delimiter2 = "-";
CurrentDate = CurrentDate.replace(delimiter2, delimiter1);
CD = CurrentDate.split(delimiter1);
date = CD[2]
month = CD[1]
year = CD[0]

In [10]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [11]:
# CHECK IF JOB IS RUNNING FOR THE FIRST TIME.
count_of_audit_ingt = pd.read_sql_query('''SELECT COUNT(1) FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN 
                                           WHERE JOB_NAME = 'TRANSACTION_SUMMARY_STG_PROCESSING' 
                                           AND JOB_STATUS = 'SUCCESS' ''',conn)
count_of_audit_ingt = count_of_audit_ingt.iloc[0,0]

In [12]:
try:      
    if ((date == '01' and month == '01') or ((date == '01' and month == '06')) or (count_of_audit_ingt == 0)):
        sql_delete_query1     = '''DELETE FROM FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_SUMMARY_STG '''
        sql_delete_query2     = '''DELETE FROM Fincore_Analytics.RISK_LKP_ENTITY_THRESHOLD where RULE_NO like 'T%' '''
        sql_delete_query3     = '''DELETE FROM FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION_rules'''
        sql_delete_query4     = '''DELETE FROM FINCORE_DB.Fincore_Analytics.RISK_TRANSACTION'''
        sql_delete_query5     = '''DELETE FROM [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_INCREMENTAL'''
        SQL_QUERY_INCREMENTAL = '''INSERT INTO [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_INCREMENTAL 
                                   SELECT REPORT_ID FROM [FINCORE_DB].[FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK) 
                                   WHERE (CPSRD.PROCESSING_FLAG_CPS2_ACCOUNT=1 AND CPSRD.PROCESSING_FLAG_CPS2_ADDRESS=1 AND 
                                   CPSRD.PROCESSING_FLAG_CPS2_CONTACT=1 AND CPSRD.PROCESSING_FLAG_CPS2_INDIVIDUAL = 1 AND 
                                   CPSRD.PROCESSING_FLAG_CPS2_ORGANIZATION=1)'''         
        cursor.execute(sql_delete_query1)
        cursor.execute(sql_delete_query2)
        cursor.execute(sql_delete_query3)
        cursor.execute(sql_delete_query4)
        cursor.execute(sql_delete_query5)
        cursor.execute(SQL_QUERY_INCREMENTAL)  
        conn.commit()
    else:        
        print("Else")
        sql_delete_query1     = '''DELETE FROM [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_INCREMENTAL'''
        SQL_QUERY_INCREMENTAL = '''INSERT INTO [FINCORE_DB].[Fincore_Analytics].RISK_TRANSACTION_INCREMENTAL 
                                   SELECT CPSRD.REPORT_ID FROM [FINCORE_DB]. [FINCORE].CPS_REPORT_DETAILS AS CPSRD WITH (NOLOCK)
                                   WHERE COALESCE(CPSRD.UPDATE_DATE,CPSRD.CREATE_DATE) > COALESCE((SELECT MAX(SOURCE_MAX_DATE) 
                                   FROM FINCORE_DB.FINCORE_ANALYTICS.AUDIT_INTGN 
                                   WHERE JOB_NAME = 'TRANSACTION_SUMMARY_STG_PROCESSING' 
                                   AND JOB_STATUS = 'SUCCESS'),CAST('1900-01-01 00:00:00.0000000' AS DATETIME2))'''
        cursor.execute(sql_delete_query1)
        cursor.execute(SQL_QUERY_INCREMENTAL)
        conn.commit()
    %run Transaction_Risk/Pop_Summary_STG.ipynb    
    
except Exception as error:
    %store -r job_id
    error = str(error)
    print("Here error")
    error = error.replace('\'','')
    error = error.replace('\"','')
    print("Failed to update record to database rollback: {}".format(error))
    conn.rollback()
    conn.execute(f'''UPDATE FINCORE_DB.Fincore_Analytics.AUDIT_INTGN set ERROR_DESC = '{error}'  where  
                     JOB_RUN_ID = {job_id}''')
    conn.commit()
    conn.close()

Stored 'job_id' (int64)
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
SUMMARY_STG_INSERTION_COMPLETED
